In [1]:
%matplotlib inline

https://indico.cern.ch/event/628669/contributions/2600995/attachments/1463252/2266733/2017_05_26_Hmm_preApproval.pdf

In [1]:
import uproot, awkward
import numpy as np
import sys
sys.path.append("/nfshome/jpata/gpu-analysis/")
if "/nfshome/jpata/.local/lib/python3.6/site-packages" in sys.path:
    sys.path.pop(sys.path.index("/nfshome/jpata/.local/lib/python3.6/site-packages"))
print(sys.path)
import hepaccelerate as ha
import os
os.environ["NUMBAPRO_NVVM"] = "/usr/local/cuda/nvvm/lib64/libnvvm.so"
os.environ["NUMBAPRO_LIBDEVICE"] = "/usr/local/cuda/nvvm/libdevice/"
#import cupy
print(os.environ["PYTHONPATH"])

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/nfshome/jpata/.ipython', '/nfshome/jpata/gpu-analysis/']


KeyError: 'PYTHONPATH'

In [13]:
awkward.__version__, cupy.__version__

NameError: name 'cupy' is not defined

In [5]:
import matplotlib.pyplot as plt

In [6]:
#[jpata@login-1 ~]$ rsync -r -R --progress /mnt/hadoop/store/./data/Run2017*/SingleMuon/NANOAOD/Nano14Dec2018-v1 titans.hep.caltech.edu:/nvmedata/store/
files = OrderedDict()
files["dy"] = glob.glob("/nvmedata/store/mc/RunIIFall17NanoAOD/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/**/*.root", recursive=True)
files["ggh"] = glob.glob("/nvmedata/store/mc/RunIIFall17NanoAOD/GluGluHToMuMu_M125_13TeV_amcatnloFXFX_pythia8/NANOAODSIM/**/*.root", recursive=True)

files["data_2017"] = glob.glob("/nvmedata/store/data/Run2017*/SingleMuon/NANOAOD/Nano14Dec2018-v1/**/*.root")

In [7]:
#remove the friend trees
files["ggh"] = [x for x in files["ggh"] if "Friend" not in x]

In [8]:
files["ggh"]

['/nvmedata/store/mc/RunIIFall17NanoAOD/GluGluHToMuMu_M125_13TeV_amcatnloFXFX_pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/20000/76434428-F342-E811-8E45-B499BAAC3786.root',
 '/nvmedata/store/mc/RunIIFall17NanoAOD/GluGluHToMuMu_M125_13TeV_amcatnloFXFX_pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/20000/0C2B3A66-B042-E811-8C6D-44A8423DE2C0.root',
 '/nvmedata/store/mc/RunIIFall17NanoAOD/GluGluHToMuMu_M125_13TeV_amcatnloFXFX_pythia8/NANOAODSIM/PU2017_12Apr2018_94X_mc2017_realistic_v14-v1/20000/E07AC933-F342-E811-A68D-3417EBE5354A.root']

In [9]:
for k, v in files.items():
    print(k, len(v))

dy 154
ggh 3
data_2017 240


In [10]:
arrays_ev = [
    "PV_npvsGood", "Flag_METFilters", "Flag_goodVertices", "Generator_weight", "genWeight", "HLT_IsoMu24"
]
arrays_jet = [
    "Jet_pt", "Jet_eta", "Jet_phi", "Jet_btagDeepB", "Jet_jetId"
]

arrays_muon = [
    "nMuon", "Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass", "Muon_pfRelIso03_all", "Muon_mediumId", "Muon_charge"
]

arrays_to_load = arrays_jet + arrays_muon

In [11]:
arrays_gen = [
    "GenPart_pt", "GenPart_eta", "GenPart_phi", "GenPart_mass", "GenPart_pdgId", "GenPart_status"
]

In [12]:
fn = files["dy"][0]
fi = uproot.open(fn)
tt = fi.get("Events")

In [13]:
"""
Preloads all data into memory.

Args:
    dataset_files (Dict[str, List[str]]): a dictionary of dataset name to the list of filenames for each dataset

returns:
    Dict[str, List[Dict[str, JaggedArray]]]
    
    For each dataset, a list of awkward-array JaggedArray dictionaries 
"""
def preload_data(dataset_files, arrays_to_load):
    ret = {}
    for dsname, dsfiles in dataset_files.items():
        ret[dsname] = [load_arrays(fn, arrays_to_load) for fn in dsfiles]
    return ret

def load_arrays(fn, arrays_to_load):
    #print("opening file {0}".format(fn))
    fi = uproot.open(fn)
    tt = fi.get("Events")
    with ThreadPoolExecutor(max_workers=16) as executor:
        arrs = tt.arrays(arrays_to_load, executor=executor)
    #print("loaded {0} arrays, shape=({1},)".format(len(arrs), tt.numentries))
    return arrs

In [14]:
def get_mem():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss/1024.0/1024.0/1024.0

In [15]:
def print_mem():
    print("Using {0:.2f} GB of RSS".format(get_mem()))

In [16]:
print_mem()
t0 = time.time()
preloaded_data = preload_data({
    "dy": files["dy"][:2],
     "ggh": files["ggh"][:2],
    "data_2017": files["data_2017"][:2]
    }, arrays_to_load)
print_mem()
t1 = time.time()
dt = t1 - t0

sum_ev = 0
for k in preloaded_data.keys():
    for arrs in preloaded_data[k]:
        sum_ev += len(arrs[b"Jet_pt"])
print("Loaded {0:.2E} events in {1:.0f} seconds, speed {2:.2E} Hz".format(sum_ev, dt, sum_ev/dt))

Using 0.19 GB of RSS
Using 3.07 GB of RSS
Loaded 8.97E+06 events in 21 seconds, speed 4.30E+05 Hz


In [41]:
pts = preloaded_data["ggh"][0][b"Jet_pt"]
etas = preloaded_data["ggh"][0][b"Jet_eta"]

class Jet:
    def __init__(self, pt, eta):
        self.pt = pt
        self.eta = eta
    def __repr__(self):
        s = "Jet({0:.2f}, {1:.2f})".format(self.pt, self.eta)
        return s

In [42]:
for iev in range(5):
    jets = []
    for imuon in range(len(pts[iev])):
        jets += [Jet(pts[iev][imuon], etas[iev][imuon])]
    print(len(jets), jets)

5 [Jet(80.88, 0.15), Jet(52.41, 0.66), Jet(38.16, -0.40), Jet(16.42, -0.37), Jet(16.28, 0.55)]
4 [Jet(87.44, 1.02), Jet(62.34, 0.78), Jet(51.72, -0.24), Jet(21.09, 1.74)]
5 [Jet(53.66, -1.67), Jet(52.50, -0.08), Jet(20.58, -0.79), Jet(16.50, 4.97), Jet(15.56, -3.47)]
4 [Jet(76.25, 0.49), Jet(62.91, 0.89), Jet(24.05, -4.15), Jet(16.22, -3.27)]
5 [Jet(130.62, -1.73), Jet(39.81, 1.18), Jet(39.12, 0.47), Jet(15.45, -3.66), Jet(15.13, 0.43)]


In [48]:
pts.offsets[:6]

array([ 0,  5,  9, 14, 18, 23])

In [49]:
pts.content[:23]

array([ 80.875    ,  52.40625  ,  38.15625  ,  16.421875 ,  16.28125  ,
        87.4375   ,  62.34375  ,  51.71875  ,  21.09375  ,  53.65625  ,
        52.5      ,  20.578125 ,  16.5      ,  15.5625   ,  76.25     ,
        62.90625  ,  24.046875 ,  16.21875  , 130.625    ,  39.8125   ,
        39.125    ,  15.4453125,  15.1328125], dtype=float32)

In [50]:
etas.content[:23]

array([ 0.14736938,  0.66174316, -0.40057373, -0.36938477,  0.54577637,
        1.0239258 ,  0.7807617 , -0.2381897 ,  1.7397461 , -1.671875  ,
       -0.07907104, -0.78637695,  4.9726562 , -3.4663086 ,  0.4921875 ,
        0.89367676, -4.1503906 , -3.270996  , -1.732666  ,  1.1765137 ,
        0.472229  , -3.661621  ,  0.42504883], dtype=float32)

In [ ]:
ds = hepaccelerate.NanoAODDataset(
    glob.glob("/nvmedata/store/mc/RunIIFall17NanoAOD/DYJetsToLL_M-50_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/**/*.root", recursive=True),
    arrays_to_load, "Events", NUMPY_LIB
)

if prepare_cache:
    ds.preload(nthreads=16, do_progress=True, event_vars=[bytes(x, encoding='ascii') for x in arrays_ev])
    ds.to_cache(do_progress=True)
else:
    ds.from_cache(do_progress=True)

ret = ds.analyze(analyze_data_function,
    is_mc=is_mc,
    pu_corrections_target=pu_corrections_2016,
    mu_pt_cut_leading=30, debug=False
)

#ret_35 = ds.analyze(analyze_data, mu_pt_cut_leading=35)
#ret_40 = ds.analyze(analyze_data, mu_pt_cut_leading=40)
ret.save_json("{0}.json".format(datasetname))

In [60]:
pts = preloaded_data["ggh"][1][b"Jet_pt"]
pts_gpu = cupy.array(pts.content)
bins_cpu = np.linspace(0,500,100)
bins_gpu = cupy.linspace(0,500,100)

In [66]:
pts.shape[0], pts.content.shape[0]

(1279071, 7590616)

In [61]:
%timeit np.histogram(pts.content, bins_cpu)

416 ms ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
%timeit cupy.histogram(pts_cuda, bins_gpu)

7.39 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
